In [1]:
from psychopy import data, visual, core, event, monitors # import some libraries from PsychoPy
import numpy as np
import psychopy.tools.monitorunittools as unittools
import pandas as pd
import time

In [17]:
# Stim Settings
ntrials    = 20
Ns         = [3,4,5,6]
sigma_s    = 4
sigma      = 4
rangemax    = 100

# Subject Settings
subID = 'test'
blockID = '1'

# Seed stimuli
N       = np.tile(Ns, int(ntrials/len(Ns)))
feeder  = np.array([1] * nCause + [0] * nUni)
mu      = np.random.normal(int(rangemax/2),sigma_s,nCause).tolist() + [None]*nUni

# Get stimuli x-values
x = list()
for i_trial in range(ntrials):
    if feeder[i_trial]==1:
        x.append(np.random.normal(mu[i_trial],sigma,[1,N[i_trial]]))
    elif feeder[i_trial]==0:
        x.append(np.random.uniform(0,100,[1,N[i_trial]]))

# Randomize vectors together
randomize = np.arange(ntrials)
np.random.shuffle(randomize)

N      = N[randomize]
feeder = feeder[randomize]
mu     = [mu[i] for i in randomize]
x      = [x[i] for i in randomize]

# Monitor settings
mon = config_monitor()

# Screen settings
screenwidth = 1440
screenheight = 900

# Scale settings
scalewidth = 800

bg_colour = [0,0,0]
win = visual.Window(
    size=[screenwidth, screenheight],
    units="pix",
    fullscr=False,
    color=bg_colour,
    colorSpace='rgb',
    mouseVisible = True,
    monitor = mon
)

# Clock settings
timer = core.Clock()

# Line settings
line = visual.Line(
    win=win,
    units="pix",
    lineColor=[-1, -1, -1]
)

# Create dictionary
data = {'Trial_ID':[],
        'Resp_loc':[], 
        'Resp_conf':[], 
        'RT_resp':[], 
        'RT_conf':[],
        'Wager':[],
        'x':[],
        'N':[],
        'x_mean':[]}

# Create dataframe
df = pd.DataFrame(data)

for trial in range(ntrials): 
    myMouse = event.Mouse(visible = True, win = win)#,newPos = [xspawn,0]) to spawn cursor at random x-coord

    resp_loc, rt_resp = get_point_response(x[trial][0])

    resp_conf, rt_conf, wager, correct = get_conf_response(x[trial][0],resp_loc)

    currtrial = pd.DataFrame({'Trial_ID':[trial+1],
                          'Resp_loc':[resp_loc],
                          'Resp_conf':[resp_conf],
                          'RT_resp':[rt_resp],
                          'RT_conf':[rt_conf],
                          'Wager':[wager],
                          'x': [x[trial]],
                          'N': [N[trial]],
                          'x_mean': [np.mean(x[trial])]})
    df = df.append(currtrial,ignore_index=True,sort=False)

        
timestr = time.strftime("%Y%m%d-%H%M%S")
filename = str('Space-S' + subID + '-B' + str(blockID) + '-' + timestr + '.csv')
print(filename)
df.to_csv(filename)

win.close()
core.quit()

Space-Stest-B1-20191016-234114.csv


SystemExit: 0

/anaconda2/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3304: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
def config_stim_settings():
    

In [3]:
def config_monitor():
    my_monitor = monitors.Monitor(name='mac_monitor')
    my_monitor.setSizePix((1280, 800))
    my_monitor.setWidth(20)
    my_monitor.setDistance(100)
    my_monitor.saveMon()
    return my_monitor

In [4]:
def config_rect_cursor(xpos, width):
    rect = visual.Rect(
        win=win,
        units="pix",
        width=width,
        height=30,
        fillColor=[1, -1, -1],
        opacity=0.3,
        pos = [x2pix(xpos),0]
    )
    return rect

In [5]:
def get_point_response(x_):
    mouse_down_detected = False
    timer.reset()
    
    while not mouse_down_detected:
        mouse_loc_pix = myMouse.getPos()
        xpos = pix2x(mouse_loc_pix[0])
        mouse_click = myMouse.getPressed()
        
        if xpos<0:
            xpos = 0
        elif xpos>100:
            xpos = 100
        
        plot_linestim(x_)
        disp_numberline()   
        
        if mouse_click[0]: # button 0 is pressed
            if not mouse_down_detected: # initial detection
                rt = timer.getTime()
                resp_loc = xpos
                mouse_down_detected = True
                        
        else: # button 0 is NOT pressed
            mouse_down_detected = False
            line = config_line(xpos,(1.0,-1,-1),200)
            line.draw()
            
            win.flip()
        
    return resp_loc, rt

In [6]:
def get_conf_response(x_,resp_loc):
    space_down_detected = False
    timer.reset()

    while not space_down_detected:
        mouse_loc_pix = myMouse.getPos()
        resp_conf_ = abs(pix2x(mouse_loc_pix[0])-resp_loc)*2
        pts = 100*np.exp(-resp_conf_/5)
        mouse_click = myMouse.getPressed()
        
        plot_linestim(x_)
        disp_numberline()
        
        keys = event.getKeys(['space'])
        if keys != []:
            if keys[0] == 'space': # space bar is pressed
                if not space_down_detected: # initial detection
                    rt        = timer.getTime()
                    resp_conf = resp_conf_
                    wager     = round(int(pts))
                    space_down_detected = True
                rect=config_rect_cursor(resp_loc,resp_conf_*scalewidth/rangemax)
                rect.draw()
                
                # plot feedback
                line = config_line(np.mean(x_),[-1, 1, -1],200)
                line.draw()

                correct = (resp_loc - resp_conf/2) < np.mean(x_) < (resp_loc + resp_conf/2)

#                 print(rect_loc)
#                 print(rect_width)
#                 print(np.mean(x_))

                if correct:
                    fb2 = '+$' + str(wager)
                    fbcolor = (-1,1,-1)
                else:
                    wager = 0
                    fb2 = '+$0'
                    fbcolor = (1,-1,-1)

                #draw_fb(fb1, (0,0,1), (x2pix(np.mean(x_))-10,60))
                draw_fb(fb2, fbcolor, (-10,-300),40)
                disp_numberline()
                win.flip() 
                core.wait(1)

        else: # space bar is NOT pressed
            space_down_detected = False
            line = config_line(resp_loc,(1.0,-1,-1),200)
            line.draw()
            
            rect=config_rect_cursor(resp_loc,resp_conf_*scalewidth/rangemax)
            rect.draw()
            pt_text = ' $'+ str(round(int(pts)))
            draw_fb(pt_text,(0,0.5,0),(-10,-300),40)
            
            win.flip()

    return resp_conf, rt, wager, correct

In [7]:
def plot_linestim(x_):
    # plot stimuli
    for offset in x_:
        line = config_line(offset,[-1,-1,-1],150)
        line.draw()

In [8]:
# def config_cross_cursor(xpos):
# # Draw cross cursor
#     size = 0.8
#     cross = visual.ShapeStim(win, 
#         vertices=((xpos, -size), (xpos, size), (xpos,0), (xpos-size,0), (xpos+size, 0)),
#         lineWidth=4,
#         closeShape=False,
#         lineColor="red"
#     )
#     return cross

In [9]:
def get_rect_response(x_, rect_locs, rect_widths, RTs):
    mouse_down_detected = False
    rect_width_pix = 10
    timer.reset()
    
    while not mouse_down_detected:
        mouse_loc_pix = unittools.deg2pix(myMouse.getPos(),mon)
        rect_loc = mouse_loc_pix[0]*(rangemax/screenwidth)+(rangemax/2)
        rect_width = rect_width_pix*(rangemax/screenwidth)
        mouse_click = myMouse.getPressed()
        if mouse_click[0]: # button 0 is pressed
            RT = timer.getTime()
            RTs.append(RT)
            
            if not mouse_down_detected: # initial detection
                rect_locs.append(rect_loc)
                rect_widths.append(rect_width)
                mouse_down_detected = True
            
            rect=config_rect_cursor(mouse_loc_pix[0],rect_width_pix)
            #cross=config_cross_cursor(myMouse.getPos()[0])
            rect.draw()
            #cross.draw()
            
            # plot stimuli
            draw_linestim(x_)
            
            # plot feedback
            line = config_line(np.mean(x_),[-1,1,-1],200)
            line.draw()
            
            correct = (rect_loc - rect_width/2) < np.mean(x_) < (rect_loc + rect_width/2)
            
            print(rect_loc)
            print(rect_width)
            print(np.mean(x_))
            
            if correct:
                pts = round(100/rect_width)
                fb = 'Correct    + ' + str(pts)
            else:
                pts = 0
                fb = 'Incorrect    + ' + str(pts)
                          
            draw_fb(fb, (-1,-1,-1), (0,0.75))
            #win.flip()
            #core.wait(1)
            
        else: # button 0 is NOT pressed
            mouse_down_detected = False
            
            keys = event.getKeys(['e','d'])
            if keys != []:
                if keys[0] == 'e':
                    rect_width_pix = rect_width_pix + 25
                elif keys[0] == 'd' and rect_width_pix > 10:
                    rect_width_pix = rect_width_pix - 25
                
            rect=config_rect_cursor(mouse_loc_pix[0],rect_width_pix)
            
            #cross=config_cross_cursor(myMouse.getPos()[0])
            rect.draw()
            #cross.draw()
            
            win.flip()
    return rect_locs, rect_widths, RTs

In [10]:
def config_line(xpos,colour,halfheight):
    line.start = [x2pix(xpos), -halfheight]
    line.end = [x2pix(xpos), +halfheight]
    line.setColor(colour,'rgb')
    #line.lineColorSpace='rgb'
    return line

In [11]:
def draw_fb(text, color, pos, height):
    text = visual.TextStim(win=win, text=text, color=color, colorSpace='rgb',units='pix', alignHoriz='left', alignVert='center', pos=pos, height=height)
    text.draw()

In [12]:
def x2pix(x):
    pix = x*scalewidth/rangemax - scalewidth/2
    return pix

def pix2x(pix):
    x = (pix+scalewidth/2)*rangemax/scalewidth
    return x    

In [13]:
def disp_numberline():
    ticks = 0,100
    for ii in ticks:
        line.start = [x2pix(ii), -30]
        line.end = [(x2pix(ii)), +30]
        line.setColor((-1, -1, 1),'rgb')
        line.draw()
        
        #draw_dispnum(ii,(x2pix(ii)-10,+30),12)
    
    #horizontal line
    line.start = [x2pix(0),0]
    line.end = [x2pix(100),0]
    line.setColor((-1, -1, 1),'rgb')
    line.draw()
    

In [14]:
Resp_locs

NameError: name 'Resp_locs' is not defined

In [24]:
test = pd.read_csv('Stest-B1-20191015-115855.csv')
test.x

0              [[19.6809803  75.41110685 27.97307412]]
1    [[ 9.98546277  2.91856925 84.77053406 42.59590...
2    [[83.95716668 75.44228761 69.54081119 29.62571...
3    [[46.90241545 23.88360081 23.42375698 72.48714...
Name: x, dtype: object

In [ ]:
N

In [ ]:
xmean

In [ ]:
Wagers

In [ ]:
Resp_locs   = []
Resp_confs  = []
RT_resp     = []
RT_conf     = []
Wagers      = []
Correct     = []

